In [ ]:
!pip install selenium

In [22]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv


In [23]:
# Configure Edge in headless mode
edge_options = Options()
edge_options.add_argument("--headless=new")
edge_options.add_argument("--disable-gpu")
edge_options.add_argument("--no-sandbox")
edge_options.add_argument("--disable-dev-shm-usage")
edge_options.add_argument("--log-level=3")
edge_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36')


driver = webdriver.Edge(options=edge_options)

# Function to read serial numbers from file
def readSerialNumber(path):
    with open(path, "r") as f:
        return [line.strip() for line in f if line.strip()]

# Function to search for warranty date by serial number
def getStatus(serial):
    driver.get("https://support.hp.com/mx-es/check-warranty")

    wait = WebDriverWait(driver, 10)

    # Wait for the form field to load
    input_element = wait.until(EC.presence_of_element_located((By.ID, 'inputtextpfinder')))
    input_element.clear()
    input_element.send_keys(serial)

    from selenium.common.exceptions import TimeoutException

    # Wait and click the accept cookies button if it appears
    try:
        privacy_accept = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
        )
        privacy_accept.click()
    except TimeoutException:
        print("Privacy notice did not appear.")

    submit_button = wait.until(EC.element_to_be_clickable((By.ID, 'FindMyProduct')))
    submit_button.click()

    # Wait for the warranty end date to appear
    try:
        warrantyStartDate = wait.until(EC.visibility_of_element_located(
            (By.XPATH, "//div[contains(@class, 'info-item')]//div[contains(text(), 'Fecha de inicio') or contains(text(), 'Start date')]/following-sibling::div"))).text

        warrantyEndDate = wait.until(EC.visibility_of_element_located(
            (By.XPATH, "//div[contains(@class, 'info-item')]//div[contains(text(), 'Fecha de expiración') or contains(text(), 'End date')]/following-sibling::div"))).text

        status = wait.until(EC.visibility_of_element_located(
            (By.XPATH, "//div[contains(@class, 'info-item')]//div[contains(text(), 'Estado') or contains(text(), 'Status')]/following-sibling::div"))).text

        return {
            "Serial": serial,
            "Start Date": warrantyStartDate,
            "End Date": warrantyEndDate,
            "Status": status
        }

    except Exception as e:
        print(f"Could not find warranty info for {serial}: {e}")
        return {"Serial": serial, "Start Date": "", "End Date": "", "Status": "No Data"}


# Path to the file with serial numbers
serial_file = "seriales.txt"

serialNumber = readSerialNumber(archivo_seriales)

results = []

for serial in serialNumber:
    print(f"Checking warranty for: {serial}")
    result = getStatus(serial)
    results.append(result)
    time.sleep(5)  # pause to avoid blocking

driver.quit()

csv_file = "warranty_results.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    fieldnames = ["Serial", "Start Date", "End Date", "Status"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in results:
        writer.writerow(row)

print(f"\nResults saved to: {csv_file}")




Consultando garantía para: 5CD350LGYG
Consultando garantía para: 5CD350LGYG
No apareció el aviso de privacidad.
